In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from backtest import backtest
from plotting import plot_fills
from downloader import Downloader, prep_config
from pure_funcs import denumpyize, numpyize, get_template_live_config, candidate_to_live_config, calc_spans, get_template_live_config, analyze_fills, \
    create_xk
from procedures import dump_live_config, load_live_config, add_argparse_args
from njit_funcs import calc_emas, calc_long_orders, round_, calc_bankruptcy_price, calc_diff
from time import time
import sys
import argparse
import pprint
import matplotlib.pyplot as plt
import json
import pandas as pd
import numpy as np

In [ ]:
plt.rcParams['figure.figsize'] = [29, 18]
pd.set_option('precision', 10)

In [ ]:
class Args:
    def __init__(self):
        self.backtest_config_path = 'configs/backtest/default.hjson'
        self.optimize_config_path = 'configs/optimize/default.hjson'
        self.exchange = 'binance'
        self.symbol = 'HBARUSDT'
        self.user = 'your_user_name'
        self.start_date = '2021-01-01'
        self.end_date = '2021-06-30'
        self.starting_balance = 100.0
config = await prep_config(Args())
dl = Downloader(config)
sts = time()
data = await dl.get_data()
prices, buyer_maker, timestamps = data
config['n_days'] = (timestamps[-1] - timestamps[0]) / (1000 * 60 * 60 * 24)

print(f'millis to load {len(prices)} ticks {(time() - sts) * 1000:.0f}ms')

In [ ]:
df = pd.DataFrame({'price': prices, 'buyer_maker': buyer_maker, 'timestamp': timestamps})

In [ ]:
# choose a slice on which to test
wsize_days = 95
ts = int(data[2][-1] - 60 * 60 * 24 * 1000 * wsize_days)
idx = np.argmax(data[2] >= ts)
dataslice = tuple(d[idx:] for d in data)

In [ ]:
template = \
    {"config_name": "hand_tuned",
     "logging_level": 0,
     "long": {"enabled": True,
              "iprc_MAr_coeffs": [[0, 0], [0, 0], [0, 0]],
              "iprc_const": 0.99,
              "iqty_MAr_coeffs": [[0, 0], [0, 0], [0, 0]],
              "iqty_const": 0.05,
              "pbr_limit": 2.0,
              "markup_MAr_coeffs": [[0, 0], [0, 0], [0, 0]],
              "markup_const": 1.005,
              "rprc_MAr_coeffs": [[0, 0], [0, 0], [0, 0]],
              "rprc_PBr_coeffs": [[-0.1, -0.15]],
              "rprc_const": 0.99,
              "rqty_MAr_coeffs": [[0, 0], [0, 0], [0, 0]],
              "rqty_const": 0.5,
              "pbr_stop_loss": 0.5},
     "max_span": 60000,
     "min_span": 6000,
     "n_spans": 3,
     "shrt": {"enabled": True,
              "iprc_MAr_coeffs": [[0, 0], [0, 0], [0, 0]],
              "iprc_const": 1.01,
              "iqty_MAr_coeffs": [[0, 0], [0, 0], [0, 0]],
              "iqty_const": 0.05,
              "pbr_limit": 2.0,
              "markup_MAr_coeffs": [[0, 0], [0, 0], [0, 0]],
              "markup_const": 0.995,
              "rprc_MAr_coeffs": [[0, 0], [0, 0], [0, 0]],
              "rprc_PBr_coeffs": [[0.1, 0.15]],
              "rprc_const": 1.01,
              "rqty_MAr_coeffs": [[0, 0], [0, 0], [0, 0]],
              "rqty_const": 0.5,
              "pbr_stop_loss": 0.5}}
config_to_test = {**config, **numpyize(template)}
dump_live_config(config_to_test, 'tmp/hand_tuned.json')

In [ ]:
sts = time()
fills, info = backtest(config_to_test, dataslice)
elapsed = time() - sts
print(f'seconds elapsed {elapsed:.4f}')
print(f'did finish {info[0]}, lowest eqbal ratio {info[1]:.4f}, closest bkr {info[2]:.4f}')
result = {**config_to_test, **{'lowest_eqbal_ratio': info[1], 'closest_bkr': info[2]}}
fdf, analysis = analyze_fills(fills, result, timestamps[0], timestamps[-1])
pprint.pprint(analysis)
fdf

In [ ]:
plot_fills(df, fdf)

In [ ]:
fdf.balance.plot()
fdf.equity.plot()

In [ ]:
fdf.pnl.sum()

In [ ]:
fdf.psize.plot()

In [ ]:
fdf.head(40)

In [ ]:
fdf.adg.plot()

In [ ]:

# plot MA bands given spans
spans = calc_spans(*[config_to_test[k] for k in ['min_span', 'max_span', 'n_spans']])
print('spans', spans)
emas = pd.DataFrame({str(span): df.price.ewm(span=span, adjust=False).mean() for span in spans})
lband = emas.min(axis=1)
uband = emas.max(axis=1)
df.price.iloc[::100].plot()
uband.iloc[::100].plot()
lband.iloc[::100].plot()


In [ ]:
# manual grid design
# modify parameters to see resulting grid
# long only

In [ ]:
xk = create_xk(config_to_test)
inverse = xk['inverse']
qty_step = xk['qty_step']
price_step = xk['price_step']
min_qty = xk['min_qty']
min_cost = xk['min_cost']
c_mult = xk['c_mult']
pbr_stop_loss = xk['pbr_stop_loss'][0]
pbr_limit = xk['pbr_limit'][0]
iqty_const = xk['iqty_const'][0]
iprc_const = xk['iprc_const'][0]
rqty_const = xk['rqty_const'][0]
rprc_const = xk['rprc_const'][0]
markup_const = xk['markup_const'][0]
iqty_MAr_coeffs = xk['iqty_MAr_coeffs'][0]
iprc_MAr_coeffs = xk['iprc_MAr_coeffs'][0]
rprc_PBr_coeffs = xk['rprc_PBr_coeffs'][0]
rqty_MAr_coeffs = xk['rqty_MAr_coeffs'][0]
rprc_MAr_coeffs = xk['rprc_MAr_coeffs'][0]
markup_MAr_coeffs = xk['markup_MAr_coeffs'][0]

In [ ]:
balance = 100.0
long_psize = 0.0
long_pprice = 0.0
highest_bid = prices[-1]
lowest_ask = prices[-1]
MA_band_lower = prices[-1]
MA_band_upper = prices[-1]
MA_ratios = np.array([1.0, 1.0, 1.0])
available_margin = 2000.0

pbr_limit = 2.0
iqty_const = 0.01
iprc_const = 0.995
rqty_const = 1.3
rprc_const = 0.98
rprc_PBr_coeffs = np.array([[-0.0, -0.05]])


orders = []

print("qty          price          psize         pprice        pprice / price ratio    pbr        bkr diff")
for k in range(10):
    long_entry, long_close = calc_long_orders(
        balance,
        long_psize,
        long_pprice,
        highest_bid,
        lowest_ask,
        MA_band_lower,
        MA_band_upper,
        MA_ratios,
        available_margin,

        inverse,
        qty_step,
        price_step,
        min_qty,
        min_cost,
        c_mult,
        pbr_stop_loss,
        pbr_limit,
        iqty_const,
        iprc_const,
        rqty_const,
        rprc_const,
        markup_const,
        iqty_MAr_coeffs,
        iprc_MAr_coeffs,
        rprc_PBr_coeffs,
        rqty_MAr_coeffs,
        rprc_MAr_coeffs,
        markup_MAr_coeffs)
    
    
    new_long_psize = long_psize + long_entry[0]
    long_pprice = long_entry[1] * (long_entry[0] / new_long_psize) + long_pprice * (long_psize / new_long_psize)
    long_psize = new_long_psize
    long_pcost = long_psize * long_pprice
    bkr_price = calc_bankruptcy_price(balance,
                          long_psize,
                          long_pprice,
                          0.0,
                          0.0,
                          inverse, c_mult)
    bkr_diff = calc_diff(bkr_price, long_pprice)
    pbr = long_pcost / balance
    if pbr > pbr_limit:
        break
    print(f"{long_entry[0]: <12} {long_entry[1]: <14} {long_psize: <12}  {round_(long_pprice, price_step): <14}"
          f"{round(long_pprice / long_entry[1], 4): <23} {round(pbr, 4): <10} {round(bkr_diff, 6)}")
    orders.append(long_entry)
print()
orders = numpyize(orders)
entry_prices = orders[:,1].astype(float)
print('grid range', entry_prices[0] / entry_prices[-1])
print('dist between grid entries', entry_prices[:-1] / entry_prices[1:])

In [ ]:
n_days_ = 14
nticks = len(df[df.timestamp > (time() - 60 * 60 * 24 * n_days_) * 1000])
print('n ticks', nticks)
edf = pd.DataFrame({p: np.repeat(p, nticks) for p in entry_prices}, index=df.index[-nticks:]).join(df.price.iloc[-nticks:])
edf.iloc[::100].plot()

In [ ]:
# load external fills to inspect

fdf = pd.read_csv('backtests/binance/BTSUSDT/plots/2021-06-12T114030/fills.csv').set_index('trade_id')

In [ ]:
longs = fdf[fdf.type.str.contains('long')]
shrts = fdf[fdf.type.str.contains('shrt')]

In [ ]:
longs

In [ ]:
longs.tail(60)

In [ ]:
plot_fills(df, fdf)

In [ ]:
fdf.balance.plot()
fdf.equity.plot()